# Watson Assistant Translation Notebook 
by: Pratyush Singh

# Watson Assistant Skill Translation 

## Installation 

In [ ]:
# After running this cell once, comment out the following code. Packages only need to be installed once.
!pip install --user --upgrade -q "ibm-watson";
!pip install --user --upgrade -q "bokeh==2.0.0";
!pip install --user --upgrade -q "pandas==1.0.1";
!pip install --user --upgrade -q "tqdm==4.43.0";

In [ ]:
import ibm_boto3
from ibm_botocore.client import Config, ClientError


In [ ]:
# Import required libraries
import pandas as pd
import json
import numpy as np
import re
from tqdm import tqdm 

from IPython.display import display
from ibm_cloud_sdk_core.authenticators import BasicAuthenticator
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson import LanguageTranslatorV3
from ibm_watson import AssistantV1


## Authentication

In [ ]:
authenticator = BasicAuthenticator('apikey', '<API_KEY_WATSON_ASSISTANT>')
sdk_object = AssistantV1(version='2020-02-05', authenticator=authenticator)
sdk_object.set_service_url('<SERVICE_URL>')

In [ ]:
assistant_information = {'workspace_id' : '<WORKSPACE_ID>',
                         'assistant_id' : '<ASSISTANT_ID>'}

In [ ]:
lt_authenticator = IAMAuthenticator('<LANGUAGE_TRANSLATOR_API_KEY')
language_translator = LanguageTranslatorV3(version='2020-04-01', authenticator=lt_authenticator)

language_translator.set_service_url('<LANGUAGE_TRANSLATOR_URL>')

In [ ]:
def translate(text:str) -> str:
    """ Translates the text into Spanish
    
    Calls the Watson Language Translator Service to translate the input text.
    
    Args: 
    - text: the input text
    
    Returns:
    - translate: the translated output
    """
    if not len(text):
        return text
    
    result = language_translator.translate(text=text, model_id='en-es').get_result()
    translate = result['translations'][0]['translation']
    
    return translate

## Intent Translation

### Get the Intents and Training Examples

In [ ]:
response = sdk_object.list_intents(workspace_id=assistant_information['workspace_id']).get_result() #handle pagination
intents = [intent['intent'] for intent in response['intents']]

In [ ]:
def _get_intent_examples(intent:str) -> str:
    """ Retrieves the intent examples that will be translated
    
    The intents for the Watson Assistant are retreived in the 
    previous cell. This function takes each intent and gets the examples. 
    It transforms the list of examples into a string seperated by commas
    
    Args:
    - intent: the intent to retrieve the examples for
    
    Returns:
    - examples_list: string of the examples joined together with a comma
    
    """
    examples = sdk_object.list_examples(workspace_id=assistant_information['workspace_id'], 
                                        intent=intent).get_result()
    examples = examples['examples']
    
    examples_list = [example['text'].replace(',', ' ') for example in examples]
    examples_list = ','.join(examples_list)
    
    return examples_list
    

In [ ]:
intents_examples = [_get_intent_examples(intent) for intent in intents]

In [ ]:
intent_df = pd.DataFrame({"intents": intents, "examples": intents_examples})
intent_df

Expands the comma seperated list into individual rows

In [ ]:
temp_df = pd.DataFrame(intent_df.examples.str.split(',').tolist(), 
                       index=intent_df.intents).stack()
temp_df = temp_df.reset_index([0, 'intents'])
temp_df.columns = ['intents', 'examples']
intents_df = temp_df.copy()

intents_df.head()

### Translate the Intent Examples 

In [ ]:
tqdm.pandas()

intents_df['translated_examples'] = intents_df['examples'].progress_apply(lambda x: translate(x))

intents_df.head()

In [ ]:
translated_intents = intents_df[['translated_examples', 'intents']]

## Translate Entities

### Retrieve Entities

In [ ]:
def get_entities() -> []:
    """ Retrieves the entities for the assistant
    """
    response = sdk_object.list_entities(workspace_id=assistant_information['workspace_id'], 
                                        export=True).get_result()
    
    return response['entities']

In [ ]:
entities = get_entities()

In [ ]:
entity_names = [entity['entity'] for entity in entities for value in entity['values'] if value['type'] == 'synonyms'] # retrieve the entity snyonyms 
not_translated = [entity['entity'] for entity in entities for value in entity['values'] if value['type'] != 'synonyms'] # keep track of the non translated entities (these are the sys entities and contextual)
values = [value['value'] for entity in entities for value in entity['values'] if value['type'] == 'synonyms'] # extract the value of the entities
synonyms = [','.join(value['synonyms']) for entity in entities for value in entity['values'] if value['type'] == 'synonyms'] # join the synonyms together in a list

In [ ]:
entities_df = pd.DataFrame({"entity": entity_names, "values": values, "synonyms": synonyms})
entities_df.head()

In [ ]:
project.save_data(file_name="Entities_English.csv", data=entities_df.to_csv(header=True, index=False, encoding='utf-8'), overwrite=True);

In [ ]:
entity_values_df = entities_df['synonyms'].str.split(',', expand=True) # tran
entity_values_df_t = entity_values_df.T
entity_values_translated_t = entity_values_df_t.applymap(lambda x: translate(x) if x is not None else x) # applyMap applies the translate function to every synonym in the DataFrame

In [ ]:
entity_values_translated_t.head()

In [ ]:
entity_values_translated = entity_values_translated_t.T
entity_values_translated.head()

In [ ]:
entity_values_translated.insert(loc=0, column='entities', value=entities_df['entity'])
entity_values_translated.insert(loc=1, column='values', value=entities_df['values'])
entity_values_translated.head()

In [ ]:
entity_values_translated['values'] = entity_values_translated['values'].progress_apply(lambda x: translate(x)) # translate the values

## Translate Dialog Nodes

This step is only section that requires a manual upload of a Data Asset. 

1. Import the skill into the workspace by clicking on *insert credentials* -- This will auto-populate the cell below with details of your COS instance
2. Next, navigate to your COS bucket where this Notebook is stored and retrieve the resource CRN.
3. In the cell below, change the variable to the name of your skill. It is best to keep the same name everytime.

In [ ]:
SKILL_NAME = ""

In [ ]:

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_1 = {
    'IAM_SERVICE_ID': '',
    'IBM_API_KEY_ID': '',
    'ENDPOINT': '',
    'IBM_AUTH_ENDPOINT': 'https://iam.cloud.ibm.com/oidc/token',
    'BUCKET': '',
    'FILE': SKILL_NAME
}

COS_RESOURCE_CRN = ""

cos = ibm_boto3.resource("s3",
    ibm_api_key_id=credentials_1['IBM_API_KEY_ID'],
    ibm_service_instance_id=COS_RESOURCE_CRN,
    ibm_auth_endpoint=credentials_1['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version="oauth"),
    endpoint_url=credentials_1['ENDPOINT']
)

In [ ]:
def get_bucket_contents(bucket_name:str, _filter:str) -> []:
    """Retrieves the skill from the COS. 
    This function searches the bucket until the file name is met.
    This function does NOT get the contents of the file, only the file path
    
    Args:
    bucket_name - name of bucket where the notebook and all data assets are stored
    _filter - name of the file
    """
    print("Retrieving bucket contents from: {0}".format(bucket_name))
    try:
        files = cos.Bucket(bucket_name).objects.all()
        logs = [file.key for file in tqdm(files, desc="Retrieving Logs from COS", 
                                          position=0, leave=True) if _filter in file.key]
        
        return logs
    
    except ClientError as be:
        print("CLIENT ERROR: {0}\n".format(be))
    except Exception as e:
        print("Unable to retrieve bucket contents: {0}".format(e))

In [ ]:
skill_file = get_bucket_contents("notebookprodforaskjac-donotdelete-pr-dh0piajhgp2k3g", SKILL_NAME)[0]

In [ ]:
def get_item(bucket_name, item_name):
    "Retrieves the contents of the file"
    try:
        file = cos.Object(bucket_name, item_name).get()
        file = file['Body'].read()
        file = file.decode("utf-8")
        file = json.loads(file)
        
        return file
    
    except ClientError as be:
        print("CLIENT ERROR: {0}\n".format(be))
    except Exception as e:
        print("Unable to retrieve file contents: {0}".format(e))


In [ ]:
skill_json = get_item("", skill_file)

In [ ]:
def replace_values(obj, key):
    """Pull all values of specified key from nested JSON."""
    text_dict = dict()

    def replace(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if k == 'options': # check for options first because we do not want to recursively translate any thing else
                    for idx, option in enumerate(v):
                        label = option['label']
                        label_trans  = translate(label)
                        option['label'] = str(label_trans)
                        v[idx] = option
                        obj[k] = v
                    continue
                if isinstance(v, (dict, list)):
                    replace(v, arr, key)
                elif k == 'title':
                    sen = translate(v)
                    obj[k] = str(sen)
                elif k == key:
                    sen = translate(v)
                    obj[k] = str(sen) # exchange/replace values. TODO: need to implement key doesn't exist case
        elif isinstance(obj, list):
            for item in obj:
                replace(item, arr, key)
        return text_dict

    results = replace(obj, text_dict, key)
    
    return results, obj  # Obj is added to be able to see dialog_nodes JSON result


In [ ]:
result = replace_values(skill_json['dialog_nodes'], 'text')

## Save 

This cell saves the translated Skills, Entities, and Intents to the Data Assets folder in Watson Studio. You can ignore this cell if you are running locally

In [ ]:
project.save_data(file_name="Translated_Skill.json", data=json.dumps(result, indent=2), overwrite=True);
project.save_data(file_name="Translated_Entities.csv", data=entity_values_translated.to_csv(header=False, index=False, encoding='utf-8'), overwrite=True);
project.save_data(file_name='Translated_Intents.csv', data=translated_intents.to_csv(header=False, index=False, encoding='utf-8'), overwrite=True);

Run the cell below, if you are running locally

In [ ]:
skill_file = open("Translated_Skill.json", "w") 
json.dump(result, out_file, indent = 6) 
skill_file.close() 

entity_values_translated.to_csv("Translated_Entities.csv", header=False, index=False, encoding='utf-8')
translated_intents.to_csv("Translated_Intents.csv", header=False, index=False, encoding='utf-8')

In [ ]:
from datetime import datetime as dt
today = dt.today().strftime("%y-%m-%d")

skill_file = f"Translated_Skill_{today}.json"
entity_file = f"Translated_Entities_{today}.csv"
intents_file = f"Translated_Intents_{today}.csv"
zip_file = f"Translated_Results_{today}.zip"

with open(skill_file, 'w') as outfile:
    json.dump(skill_json, outfile, indent=2)

entity_values_translated.to_csv(entity_file, header=False, index=False, encoding='utf-8')
translated_intents.to_csv(intents_file, header=False, index=False, encoding='utf-8')

In [ ]:
from zipfile import ZipFile

with ZipFile(zip_file, 'w') as zip: 
        # writing each file one by one 
        for file in [skill_file, entity_file, intents_file]: 
            zip.write(file) 

Run the cell below if you are running on Watson Studio

In [ ]:
import os
def copy_python_env_file_to_project_asset(project=project, python_env_source_dir='python environemnt source directory', python_env_filename='python environment file name'):
    filename_with_path=python_env_source_dir+'/'+python_env_filename
    print('Source directory listing: ')
    print(os.listdir(python_env_source_dir))
    print('File: ')
    print(os.stat(filename_with_path))
    print('Copying file {0} from python environment to project assets'.format(python_env_filename))
    file_data=open(filename_with_path, 'rb')
    project.save_data(data=file_data.read(),file_name=python_env_filename,overwrite=True)
    file_data.close() 
    print('Done.')

In [ ]:
copy_python_env_file_to_project_asset(project=project, python_env_source_dir='.', python_env_filename=zip_file)